# Bayesian Network Tutorial

## Part 1: Bayesian Network Tutorial

To start, design a basic probabilistic model for the following system:

James Bond, Q (Quartermaster), and M (Head of MI6) are in charge of the security system at MI6, the Military Intelligence Unit of the British Secret Service. MI6 runs a special program called “Double-0”, where secret spy agents are trained and deployed to gather information concerning national security. A terrorist organization named “Spectre” is planning an espionage mission and its aim is to gain access to the secret “Double-0” files stored in the MI6 database. Q has designed a special security system to protect the secret “Double-0” files. In order to gain access to these files, Spectre needs to steal from MI6 a cipher and the key to crack this cipher. Q stores this cipher in his personal database, which is guarded by heavy security protocols. The key to cracking the cipher is known only to M, who is protected by Bond.

### Steps Spectre Can Take:

- **Hire professional hackers** who can write programs to launch a cyberattack on Q’s personal database.
- **Buy a state-of-the-art computer called “Contra”** to actually launch this cyberattack.
- **Hire ruthless mercenaries** to kidnap M and get access to the key.
- **Ensure Bond is not available** with M at the time of the kidnapping.
- **Use the cipher and key** to access the target “Double-0” files.

### Variables:

- **H**: Spectre hires professional hackers.
- **C**: Spectre buys Contra.
- **M**: Spectre hires mercenaries.
- **B**: Bond is guarding M at the time of the kidnapping.
- **Q**: Q’s database is hacked and the cipher is compromised.
- **K**: M gets kidnapped and has to give away the key.
- **D**: Spectre succeeds in obtaining the “Double-0” files.

### Classified Information:

- Spectre fails to hire skilled professional hackers with a probability of 0.5.
- Spectre obtains Contra with a probability of 0.3.
- Spectre fails to hire mercenaries with a probability of 0.2.
- Bond is protecting M with a probability of 0.5.
- If professional hackers are hired:
  - They can crack Q's database without Contra with a probability of 0.55.
  - With Contra, they can crack Q's database with a probability of 0.9.
- If professional hackers are not hired:
  - Less experienced employees attempt the hack.
  - Q's database remains secure with a probability of 0.75 with Contra.
  - Q's database remains secure with a probability of 0.95 without Contra.
- When Bond is protecting M:
  - M stays safe with a probability of 0.85 if mercenaries attack.
  - M stays safe with a probability of 0.99 if mercenaries do not attack.
- When Bond is not protecting M:
  - M gets kidnapped with a probability of 0.95 with mercenaries.
  - M gets kidnapped with a probability of 0.75 without mercenaries.
- With both the cipher and the key, Spectre accesses the “Double-0” files with a probability of 0.99.
- With neither, the probability drops to 0.02.
- With just the cipher, the probability is 0.4.
- With just the key, the probability is 0.65.

### 1a: Casting the Net

Use the description above to design a Bayesian network for this model. We'll use the `pgmpy` package to represent nodes and conditional probability arcs connecting nodes.

In [ ]:
# Import necessary libraries
from pgmpy.models import BayesianNetwork

def make_security_system_net():
    # Initialize the Bayesian Network
    BayesNet = BayesianNetwork()
    
    # Add nodes
    BayesNet.add_nodes_from(['H', 'C', 'M', 'B', 'Q', 'K', 'D'])
    
    # Add edges
    # Edges for Q's database being hacked
    BayesNet.add_edge('H', 'Q')
    BayesNet.add_edge('C', 'Q')
    
    # Edges for M getting kidnapped
    BayesNet.add_edge('M', 'K')
    BayesNet.add_edge('B', 'K')
    
    # Edge for Spectre obtaining the files
    BayesNet.add_edge('Q', 'D')
    BayesNet.add_edge('K', 'D')
    
    return BayesNet

# Create the Bayesian network
bayes_net = make_security_system_net()

### 1b: Setting the Probabilities

Now, set the conditional probabilities for the necessary variables on the network you just built.

In [ ]:
from pgmpy.factors.discrete import TabularCPD

def set_probability(bayes_net):
    # Prior probabilities
    cpd_H = TabularCPD('H', 2, [[0.5], [0.5]])  # P(H)
    cpd_C = TabularCPD('C', 2, [[0.7], [0.3]])  # P(C)
    cpd_M = TabularCPD('M', 2, [[0.2], [0.8]])  # P(M)
    cpd_B = TabularCPD('B', 2, [[0.5], [0.5]])  # P(B)
    
    # Conditional probabilities for Q
    cpd_Q = TabularCPD('Q', 2, 
                       values=[
                           [0.95, 0.75, 0.45, 0.10],  # P(Q=False)
                           [0.05, 0.25, 0.55, 0.90]   # P(Q=True)
                       ],
                       evidence=['H', 'C'],
                       evidence_card=[2, 2])
    
    # Conditional probabilities for K
    cpd_K = TabularCPD('K', 2, 
                       values=[
                           [0.99, 0.85, 0.25, 0.05],  # P(K=False)
                           [0.01, 0.15, 0.75, 0.95]   # P(K=True)
                       ],
                       evidence=['B', 'M'],
                       evidence_card=[2, 2])
    
    # Conditional probabilities for D
    cpd_D = TabularCPD('D', 2, 
                       values=[
                           [0.98, 0.60, 0.35, 0.01],  # P(D=False)
                           [0.02, 0.40, 0.65, 0.99]   # P(D=True)
                       ],
                       evidence=['Q', 'K'],
                       evidence_card=[2, 2])
    
    # Add CPDs to the network
    bayes_net.add_cpds(cpd_H, cpd_C, cpd_M, cpd_B, cpd_Q, cpd_K, cpd_D)
    
    # Check if the model is valid
    assert bayes_net.check_model(), "Bayesian network model is invalid."

# Set the probabilities
set_probability(bayes_net)

### 1c: Probability Calculations - Perform Inference

Now, perform inference on the network to calculate the following probabilities:

1. What is the **marginal probability** that the “Double-0” files get compromised?
2. Given that **Contra is unavailable**, what is the **conditional probability** that the “Double-0” files get compromised?
3. Given that **Contra is unavailable** and **Bond is guarding M**, what is the **conditional probability** that the “Double-0” files get compromised?

In [ ]:
from pgmpy.inference import VariableElimination

def get_marginal_double0(bayes_net):
    solver = VariableElimination(bayes_net)
    marginal_prob = solver.query(variables=['D'], joint=False)
    prob = marginal_prob['D'].values
    # Return the probability that D=True (files get compromised)
    return prob[1]

def get_conditional_double0_given_no_contra(bayes_net):
    solver = VariableElimination(bayes_net)
    conditional_prob = solver.query(variables=['D'], evidence={'C': 0}, joint=False)
    prob = conditional_prob['D'].values
    # Return the probability that D=True given C=False
    return prob[1]

def get_conditional_double0_given_no_contra_and_bond_guarding(bayes_net):
    solver = VariableElimination(bayes_net)
    conditional_prob = solver.query(variables=['D'], evidence={'C': 0, 'B': 1}, joint=False)
    prob = conditional_prob['D'].values
    # Return the probability that D=True given C=False and B=True
    return prob[1]

# Calculate the probabilities
prob_marginal = get_marginal_double0(bayes_net)
prob_conditional_no_contra = get_conditional_double0_given_no_contra(bayes_net)
prob_conditional_no_contra_bond_guarding = get_conditional_double0_given_no_contra_and_bond_guarding(bayes_net)

print(f"Marginal Probability that the 'Double-0' files get compromised: {prob_marginal:.4f}")
print(f"Conditional Probability given Contra is unavailable: {prob_conditional_no_contra:.4f}")
print(f"Conditional Probability given Contra is unavailable and Bond is guarding M: {prob_conditional_no_contra_bond_guarding:.4f}")

### Interpretation:

1. **Marginal Probability**: There's approximately a 41.77% chance that the "Double-0" files get compromised under normal conditions.
2. **Without Contra**: The probability drops to approximately 26.34% when Contra is unavailable.
3. **Without Contra & Bond Guarding**: The probability further decreases to approximately 15.05% when Bond is also guarding M.

## Conclusion

By modeling the security scenario with a Bayesian network, we've identified how different factors influence the probability of the "Double-0" files being compromised. Proactive actions like disabling Contra and ensuring Bond guards M significantly reduce the risk.